# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
import pandas as pd
import sqlite3

In [2]:
# Connect to the database
conn = sqlite3.connect('data.sqlite')

# Create a cursor object to interact with the database
cursor = conn.cursor()

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:

# Define the SQL query
query = '''
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
)
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql(query, conn)

# Display the DataFrame
print(df)


   customerNumber contactLastName contactFirstName
0             141          Freyre           Diego 


### In this query, the subquery (SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31') is used to retrieve the customer numbers for orders placed on January 31, 2003. The outer query then selects the customer 

### By using the IN operator with the subquery, we can filter the rows from the customers table based on the customer numbers returned by the subquery. This achieves the same result as the original query with a JOIN.

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
import pandas as pd
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.sqlite')

# Define the SQL query
query = '''
SELECT
    p.productName,
    (
        SELECT SUM(od.quantityOrdered)
        FROM orderDetails od
        WHERE od.productCode = p.productCode
    ) AS totalItemsSold
FROM products p
ORDER BY totalItemsSold DESC
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql(query, conn)

# Display the DataFrame
df


,productName,totalItemsSold
0,1992 Ferrari 360 Spider red,1808.0
1,1937 Lincoln Berline,1111.0
2,American Airlines: MD-11S,1085.0
3,1941 Chevrolet Special Deluxe Cabriolet,1076.0
4,1930 Buick Marquette Phaeton,1074.0
...,...,...
105,1911 Ford Town Car,832.0
106,1936 Mercedes Benz 500k Roadster,824.0
107,1970 Chevy Chevelle SS 454,803.0
108,1957 Ford Thunderbird,767.0


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [5]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT


# Define the SQL query
query = '''
SELECT
    p.productName,
    (
        SELECT COUNT(DISTINCT o.customerNumber)
        FROM orders o
        JOIN orderDetails od 
            ON o.orderNumber = od.orderNumber
        WHERE od.productCode = p.productCode
    ) AS totalCustomers
FROM products p
ORDER BY totalCustomers DESC
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql(query, conn)

# Display the DataFrame
print(df)


                     productName  totalCustomers
0    1992 Ferrari 360 Spider red              40
1       1952 Alpine Renault 1300              27
2            1972 Alfa Romeo GTA              27
3             1934 Ford V8 Coupe              27
4               Boeing X-32A JSF              27
..                           ...             ...
105           1949 Jaguar XK 120              18
106            1952 Citroen-15CV              18
107    1969 Chevrolet Camaro Z28              18
108          2002 Chevy Corvette              18
109            1985 Toyota Supra               0

[110 rows x 2 columns]


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [6]:

# Define the SQL query
query = '''
SELECT DISTINCT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    o.city,
    e.officeCode
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
WHERE e.employeeNumber IN (
    SELECT DISTINCT e.employeeNumber
    FROM employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders o ON c.customerNumber = o.customerNumber
    JOIN orderDetails od ON o.orderNumber = od.orderNumber
    WHERE od.productCode IN (
        SELECT p.productCode
        FROM products p
        JOIN orderDetails od ON p.productCode = od.productCode
        JOIN orders o ON od.orderNumber = o.orderNumber
        GROUP BY od.productCode
        HAVING COUNT(DISTINCT o.customerNumber) < 20
    )
)
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql(query, conn)

# Display the DataFrame
print(df)


    employeeNumber firstName   lastName           city  officeCode
0             1165    Leslie   Jennings  San Francisco           1
1             1166    Leslie   Thompson  San Francisco           1
2             1188     Julie   Firrelli         Boston           2
3             1216     Steve  Patterson         Boston           2
4             1286  Foon Yue      Tseng            NYC           3
5             1323    George     Vanauf            NYC           3
6             1337      Loui     Bondur          Paris           4
7             1370    Gerard  Hernandez          Paris           4
8             1401    Pamela   Castillo          Paris           4
9             1501     Larry       Bott         London           7
10            1504     Barry      Jones         London           7
11            1611      Andy     Fixter         Sydney           6
12            1612     Peter      Marsh         Sydney           6
13            1621      Mami      Nishi          Tokyo        

### To solve this problem, we can break it down into the following steps:

1. Identify the product codes that have been ordered by fewer than 20 people.
2. Find the employees who have sold those products.
3. Retrieve the employee number, first name, last name, city (of the office), and office code for each employee.
4. Ensure that each employee is listed only once.

Here's the query to accomplish these steps using aliases and subqueries:

```python
import pandas as pd
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.sqlite')

# Define the SQL query
query = '''
SELECT DISTINCT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    o.city,
    e.officeCode
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
WHERE e.employeeNumber IN (
    SELECT DISTINCT e.employeeNumber
    FROM employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders o ON c.customerNumber = o.customerNumber
    JOIN orderDetails od ON o.orderNumber = od.orderNumber
    WHERE od.productCode IN (
        SELECT p.productCode
        FROM products p
        JOIN orderDetails od ON p.productCode = od.productCode
        JOIN orders o ON od.orderNumber = o.orderNumber
        GROUP BY od.productCode
        HAVING COUNT(DISTINCT o.customerNumber) < 20
    )
)
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)
```

In this query, we use multiple subqueries to achieve the desired result. 

The innermost subquery `(SELECT p.productCode FROM products p JOIN orderDetails od ON p.productCode = od.productCode JOIN orders o ON od.orderNumber = o.orderNumber GROUP BY od.productCode HAVING COUNT(DISTINCT o.customerNumber) < 20)` retrieves the product codes that have been ordered by fewer than 20 people.

The next subquery `(SELECT DISTINCT e.employeeNumber FROM employees e JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber JOIN orders o ON c.customerNumber = o.customerNumber JOIN orderDetails od ON o.orderNumber = od.orderNumber WHERE od.productCode IN (...))` identifies the employee numbers of those employees who have sold the products from the previous subquery.

Finally, the main query retrieves the employee number, first name, last name, city (of the office), and office code for each employee from the `employees` and `offices` tables. The `DISTINCT` keyword is used to ensure that each employee is listed only once.

The query results are fetched into a Pandas DataFrame using `pd.read_sql()`, and then the DataFrame is printed to display the retrieved data.

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [7]:
# Define the SQL query
query = '''
SELECT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    COUNT(c.customerNumber) AS numCustomers
FROM
    employees e
JOIN
    customers c ON e.employeeNumber = c.salesRepEmployeeNumber
WHERE
    c.customerNumber IN (
        SELECT
            customerNumber
        FROM
            customers
        GROUP BY
            salesRepEmployeeNumber
        HAVING
            AVG(creditLimit) > 15000
    )
GROUP BY
    e.employeeNumber,
    e.firstName,
    e.lastName
'''

# Execute the query and fetch the results into a DataFrame
df = pd.read_sql(query, conn)


# Display the DataFrame
print(df)


    employeeNumber firstName   lastName  numCustomers
0             1165    Leslie   Jennings             1
1             1166    Leslie   Thompson             1
2             1188     Julie   Firrelli             1
3             1216     Steve  Patterson             1
4             1286  Foon Yue      Tseng             1
5             1323    George     Vanauf             1
6             1337      Loui     Bondur             1
7             1370    Gerard  Hernandez             1
8             1401    Pamela   Castillo             1
9             1501     Larry       Bott             1
10            1504     Barry      Jones             1
11            1611      Andy     Fixter             1
12            1612     Peter      Marsh             1
13            1621      Mami      Nishi             1
14            1702    Martin     Gerard             1


## The steps involved in the query:

1. Join the `employees` and `customers` tables on the `employeeNumber` and `salesRepEmployeeNumber` columns, respectively.
2. In the subquery `(SELECT c2.customerNumber FROM customers c2 GROUP BY c2.customerNumber HAVING AVG(c2.creditLimit) > 15000)`, group the customers by their customer number and filter them based on their average credit limit being greater than 15,000.
3. Use the `WHERE` clause in the main query to check if the customer number of each customer is present in the subquery results. This filters out the employees whose customers have an average credit limit over 15,000.
4. Use the `GROUP BY` clause to group the results by the employee number.
5. Use the `COUNT()` function to calculate the number of customers for each employee.
6. Select the employee number, first name, last name, and the count of customers as `numCustomers` in the final result set.

The query results are fetched into a Pandas DataFrame using `pd.read_sql()`, and then the DataFrame is printed to display the retrieved data.

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!

In [8]:
# Close the database connection
conn.close()